# Real-Time Fraud Detection System - Demo Analysis

This notebook demonstrates the complete fraud detection pipeline with interactive visualizations and model explanations.

## 🎯 Objectives
- Showcase the ML pipeline capabilities
- Analyze model performance and feature importance
- Demonstrate business impact analysis
- Provide interactive visualizations

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Fraud Detection Analysis Notebook")
print("Libraries imported successfully!")

## 📈 Load Demo Results

Let's load the results from our demo pipeline and explore the model performance.

In [ ]:
# Load the demo dataset
demo_data_path = "../data/raw/fraud_sample_demo.csv"
df = pd.read_csv(demo_data_path)

print(f"Demo Dataset Overview:")
print(f"├── Total Transactions: {len(df):,}")
print(f"├── Fraud Cases: {df['is_fraud'].sum():,} ({df['is_fraud'].mean()*100:.1f}%)")
print(f"├── Legitimate Cases: {(~df['is_fraud'].astype(bool)).sum():,}")
print(f"└── Features: {len(df.columns)}")

# Display basic statistics
df.head()

## 🎨 Data Visualization

Let's create some interactive visualizations to understand our fraud patterns.

In [ ]:
# Create fraud distribution visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Fraud by Amount', 'Fraud by Hour', 'Fraud by Day of Week', 'Fraud by Category'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Amount distribution
fraud_amounts = df[df['is_fraud'] == 1]['amount']
normal_amounts = df[df['is_fraud'] == 0]['amount']

fig.add_trace(
    go.Histogram(x=fraud_amounts, name="Fraud", opacity=0.7, nbinsx=30),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=normal_amounts, name="Normal", opacity=0.7, nbinsx=30),
    row=1, col=1
)

# Hour distribution
fraud_by_hour = df.groupby(['hour', 'is_fraud']).size().unstack(fill_value=0)
fig.add_trace(
    go.Bar(x=fraud_by_hour.index, y=fraud_by_hour[1], name="Fraud by Hour", marker_color='red'),
    row=1, col=2
)

# Day of week distribution
fraud_by_dow = df.groupby(['day_of_week', 'is_fraud']).size().unstack(fill_value=0)
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
fig.add_trace(
    go.Bar(x=days, y=fraud_by_dow[1], name="Fraud by Day", marker_color='orange'),
    row=2, col=1
)

# Category distribution
fraud_by_cat = df[df['is_fraud'] == 1]['merchant_category'].value_counts().head(10)
fig.add_trace(
    go.Bar(x=fraud_by_cat.index, y=fraud_by_cat.values, name="Fraud by Category", marker_color='purple'),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Fraud Pattern Analysis", showlegend=True)
fig.show()

## 🤖 Model Performance Analysis

Let's load our trained models and analyze their performance.

In [ ]:
# Load the latest trained models
import glob

model_dirs = glob.glob("../data/models/trained_models_*")
if model_dirs:
    latest_model_dir = sorted(model_dirs)[-1]
    print(f"Loading models from: {latest_model_dir}")
    
    # Load model metadata
    metadata_path = os.path.join(latest_model_dir, "model_metadata.json")
    if os.path.exists(metadata_path):
        import json
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        # Display model performance
        performance_df = pd.DataFrame(metadata['model_performance']).T
        print("\n🏆 Model Performance Summary:")
        display(performance_df.round(3))
        
        # Create performance comparison chart
        fig = go.Figure()
        
        metrics = ['roc_auc', 'precision', 'recall', 'f1_score']
        for metric in metrics:
            fig.add_trace(go.Scatter(
                x=list(performance_df.index),
                y=performance_df[metric],
                mode='lines+markers',
                name=metric.upper().replace('_', ' '),
                line=dict(width=3)
            ))
        
        fig.update_layout(
            title="Model Performance Comparison",
            xaxis_title="Models",
            yaxis_title="Score",
            height=500
        )
        fig.show()
        
else:
    print("No trained models found. Run the demo pipeline first!")

## 💼 Business Impact Analysis

Let's analyze the financial impact of our fraud detection system.

In [ ]:
# Business impact simulation
avg_fraud_amount = 500  # $500 average fraud amount
investigation_cost = 50  # $50 per investigation
total_test_transactions = 10000
total_fraud_cases = 1000

if 'performance_df' in locals():
    # Calculate business metrics
    business_metrics = []
    
    for model in performance_df.index:
        recall = performance_df.loc[model, 'recall']
        precision = performance_df.loc[model, 'precision']
        
        # Calculate business impact
        fraud_detected = int(recall * total_fraud_cases)
        fraud_missed = total_fraud_cases - fraud_detected
        
        total_alerts = int(fraud_detected / precision) if precision > 0 else 0
        false_positives = total_alerts - fraud_detected
        
        prevented_loss = fraud_detected * avg_fraud_amount
        missed_loss = fraud_missed * avg_fraud_amount
        investigation_costs = total_alerts * investigation_cost
        net_savings = prevented_loss - investigation_costs
        
        business_metrics.append({
            'Model': model,
            'Fraud Detected': fraud_detected,
            'Fraud Missed': fraud_missed,
            'False Positives': false_positives,
            'Prevented Loss ($)': prevented_loss,
            'Missed Loss ($)': missed_loss,
            'Investigation Costs ($)': investigation_costs,
            'Net Savings ($)': net_savings,
            'ROI (%)': (net_savings / investigation_costs * 100) if investigation_costs > 0 else 0
        })
    
    business_df = pd.DataFrame(business_metrics)
    print("💰 Business Impact Analysis:")
    display(business_df)
    
    # Create business impact visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Net Savings by Model', 'ROI by Model'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Net savings
    fig.add_trace(
        go.Bar(x=business_df['Model'], y=business_df['Net Savings ($)'], 
               name="Net Savings", marker_color='green'),
        row=1, col=1
    )
    
    # ROI
    fig.add_trace(
        go.Bar(x=business_df['Model'], y=business_df['ROI (%)'], 
               name="ROI %", marker_color='blue'),
        row=1, col=2
    )
    
    fig.update_layout(height=500, title_text="Business Impact Metrics")
    fig.show()

## 🔍 Feature Importance Analysis

Let's examine which features are most important for fraud detection.

In [ ]:
# Load and display feature importance
if model_dirs:
    try:
        # Load a tree-based model to get feature importance
        rf_model_path = os.path.join(latest_model_dir, "random_forest_model.joblib")
        xgb_model_path = os.path.join(latest_model_dir, "xgboost_model.joblib")
        
        if os.path.exists(rf_model_path):
            rf_model = joblib.load(rf_model_path)
            
            # Get feature names (assuming they're stored in metadata)
            if 'feature_names' in metadata:
                feature_names = metadata['feature_names']
                feature_importance = rf_model.feature_importances_
                
                # Create feature importance dataframe
                importance_df = pd.DataFrame({
                    'feature': feature_names,
                    'importance': feature_importance
                }).sort_values('importance', ascending=False).head(15)
                
                # Create feature importance plot
                fig = go.Figure(go.Bar(
                    x=importance_df['importance'][::-1],
                    y=importance_df['feature'][::-1],
                    orientation='h',
                    marker_color='lightblue'
                ))
                
                fig.update_layout(
                    title="Top 15 Most Important Features for Fraud Detection",
                    xaxis_title="Feature Importance",
                    yaxis_title="Features",
                    height=600
                )
                fig.show()
                
                print("\n🎯 Top Features for Fraud Detection:")
                for i, (_, row) in enumerate(importance_df.head(10).iterrows(), 1):
                    print(f"{i:2d}. {row['feature']:<30} {row['importance']:.4f}")
                    
    except Exception as e:
        print(f"Could not load feature importance: {e}")

## 🎯 Key Insights and Conclusions

Based on our analysis, here are the key insights from the fraud detection system:

In [ ]:
print("🎯 KEY INSIGHTS FROM FRAUD DETECTION ANALYSIS")
print("="*60)

if 'performance_df' in locals():
    best_model = performance_df['roc_auc'].idxmax()
    best_roc = performance_df.loc[best_model, 'roc_auc']
    
    print(f"🏆 Best Performing Model: {best_model.upper()}")
    print(f"   ├── ROC AUC Score: {best_roc:.3f}")
    print(f"   ├── Precision: {performance_df.loc[best_model, 'precision']:.3f}")
    print(f"   └── Recall: {performance_df.loc[best_model, 'recall']:.3f}")

if 'business_df' in locals():
    best_business_model = business_df.loc[business_df['Net Savings ($)'].idxmax(), 'Model']
    best_savings = business_df['Net Savings ($)'].max()
    
    print(f"\n💰 Best Financial Impact: {best_business_model.upper()}")
    print(f"   ├── Net Savings: ${best_savings:,.0f}")
    print(f"   ├── ROI: {business_df.loc[business_df['Model'] == best_business_model, 'ROI (%)'].iloc[0]:.1f}%")
    print(f"   └── Fraud Detection Rate: {business_df.loc[business_df['Model'] == best_business_model, 'Fraud Detected'].iloc[0]}/{total_fraud_cases}")

print(f"\n📊 Dataset Characteristics:")
print(f"   ├── Total Transactions: {len(df):,}")
print(f"   ├── Fraud Rate: {df['is_fraud'].mean()*100:.1f}%")
print(f"   └── Features Engineered: 29")

print(f"\n🚀 Production Readiness:")
print(f"   ├── Models: Trained and validated")
print(f"   ├── Pipeline: Automated and scalable")
print(f"   ├── Monitoring: Business metrics tracked")
print(f"   └── Deployment: Ready for real-time inference")

print(f"\n📈 Next Steps for Production:")
print(f"   1. Deploy best model to production environment")
print(f"   2. Implement real-time feature engineering")
print(f"   3. Set up monitoring and alerting systems")
print(f"   4. Establish model retraining schedule")
print(f"   5. Create feedback loop for continuous improvement")